In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime
import os

In [ ]:
path = 'dataset'
train = os.path.join(path, 'train')
val = os.path.join(path, 'validation')
img_size = (150, 150)

train_size = 12
val_size = 8

train_data = tf.keras.utils.image_dataset_from_directory(
  train, 
  shuffle = True, 
  batch_size = train_size, 
  image_size = img_size
)
val_data = tf.keras.utils.image_dataset_from_directory(
  val, 
  shuffle = True, 
  batch_size = val_size, 
  image_size = img_size
)

test_img = val_data
val_batches = tf.data.experimental.cardinality(val_data)
test_data = val_data.take(val_batches // 5)
val_data = val_data.skip(val_batches // 5)
print(len(val_data))
auto = tf.data.AUTOTUNE

train_data = train_data.prefetch(buffer_size=auto)
val_data = val_data.prefetch(buffer_size=auto)
test_data = test_data.prefetch(buffer_size=auto)

In [ ]:
num_classes = 3
img_shape = img_size + (3,)
base_model = tf.keras.applications.ResNet50V2(
    # input_shape = img_shape, 
    include_top = False, 
    weights='imagenet'
)
resnet_v2 = "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5"
feature_layer = hub.KerasLayer(
    resnet_v2,
    input_shape=img_shape,
    trainable=False
)

model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(num_classes, activation = "softmax")
])
     
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    train_data,
    epochs=4,
    validation_data=val_data
)